<a href="https://colab.research.google.com/github/amanotk/numerical-geophysics/blob/main/notebook/LinearAdvectionEquation.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab">
</a>

# 線形移流方程式

以下の線形移流方程式の数値解を差分法によって求める．
$$
\frac{\partial u}{\partial t} + c \frac{\partial u}{\partial x} = 0
$$
ただし$c = 1$，$0 \leq x \leq 1$とする．

- 境界条件（$x = 0, 1$）  
周期境界条件
$$
u(1) = u(0)
$$
を採用する．
- 初期条件（$t = 0$）  
$$
u(x) =
\begin{cases}
& 1 \quad 0 \leq x < \frac{1}{2}
\\
& 0 \quad \frac{1}{2} \leq x < 1
\end{cases}
$$
または
$$
u(x) = \sin \left( 2\pi x \right)
$$
のいずれか．

以下ではグリッド数を$N_x$, グリッド幅を$\Delta x = 1/N_x$，時間ステップを$\Delta t$とする．また，Courant数を$\nu = c \Delta t / \Delta x$と置く．

## テクニカルな話

### 時間発展の実装

スキームの説明に$u^{n}_{i}$のような書き方がされるので時間方向にもメモリを確保して2次元配列を使おうとする人が度々見受けられるが，通常はそのようなことはしない．時間発展の計算には過去のデータは必要ないので，配列を新しい解で次々上書きしていけばよい．  
得られた数値解の時間発展を解析したい場合には必要な時間ステップのデータのみファイルに保存しておけき，計算が終了した後に解析をする．（ただし，このNotebookではデータの保存はせずにプロットだけをしている．）


### 境界条件の指定方法

差分法系のスキームの実装には，数値解を格納する配列として物理領域よりも境界条件の分だけ余分にメモリを確保して用いる．例えばグリッド数`Nx`に対して配列の長さを`Nx+2`とし，物理領域の配列`u[ix]`のインデックスは`ix = 1, 2, ..., Nx`までとする．

例えばFTCSスキームで`u[1]`の解を更新するには`u[0]`の値が，`u[Nx]`の値を更新するには`u[Nx+1]`の値がそれぞれ適切に設定されている必要がある．毎ステップこれらの値を境界条件に基づいて適切に設定しておけば，`ix = 1, 2, ..., Nx`の全ての点について同じ手順で`u[ix]`を更新できる（無駄な条件分岐などが必要ない）．具体的に，周期境界条件の場合は
```python
# 境界条件
u[   0] = u[Nx]
u[Nx+1] = u[ 1]
```
のようにすればよい．

並列計算をしなければならない場合にはこのような実装がほぼ必須であり，境界条件を設定する`u[0]`や`u[Nx+1]`を（「のりしろ」と呼ばれる）を異なるプロセス間で転送することになる．

In [ ]:
# 準備
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt

In [ ]:
# フォントサイズ
plt.rcParams["font.size"] = 14

# 初期条件
def set_initial(Nx, type=0):
  "初期条件を設定する(typeによって切り替える）"
  dx = 1/Nx
  xx = (np.arange(Nx+2) - 0.5)*dx
  if type == 0:
    return xx, np.where(xx<0.5, 1.0, 0.0)  
  if type == 1:
    return xx, np.sin(2*np.pi*xx)

## FTCSスキーム

$$
u^{n+1}_{i} = u^{n}_{i} - \frac{1}{2} \nu \left( u^{n}_{i+1} - u^{n}_{i-1} \right)
$$

In [ ]:
def push_ftcs(u, nu, step):
  "FTCSスキームによってstep数だけ時間更新する"
  Nx = np.size(u) - 2
  ix = np.arange(1, Nx+1, dtype=np.int32)
  for n in range(step):
    # 更新
    u[ix] = u[ix] - 0.5*nu*(u[ix+1] - u[ix-1])
    # 境界条件
    u[   0] = u[Nx]
    u[Nx+1] = u[ 1]

# パラメータ
Nx = 50
nu = 0.5
dx = 1/Nx
dt = nu * dx
ix = np.arange(1, Nx+1)

# 初期条件
x, u = set_initial(Nx, 0)

# プロット
fig, axs = plt.subplots(figsize=(10, 6))
step = 5
for n in range(4):
  plt.plot(x[ix], u[ix], label='t = {:5.3f}'.format(n*step*dt))
  push_ftcs(u, nu, step)

plt.xlim(0.0, 1.0)
plt.xlabel('x')
plt.ylabel('u')
plt.legend(loc='upper left', bbox_to_anchor=(1.0, 1.0))
plt.suptitle('FTCS')

## FTFSスキームおよびFTBSスキーム

- FTFSスキーム
$$
u^{n+1}_{i} = u^{n}_{i} - \nu \left( u^{n}_{i+1} - u^{n}_{i} \right)
$$

- FTBSスキーム
$$
u^{n+1}_{i} = u^{n}_{i} - \nu \left( u^{n}_{i} - u^{n}_{i-1} \right)
$$

In [ ]:
def push_ftfs(u, nu, step):
  "FTFSスキームによってstep数だけ時間更新する"
  Nx = np.size(u) - 2
  ix = np.arange(1, Nx+1, dtype=np.int32)
  for n in range(step):
    # 更新
    u[ix] = u[ix] - nu*(u[ix+1] - u[ix])
    # 境界条件
    u[   0] = u[Nx]
    u[Nx+1] = u[ 1]

# パラメータ
Nx = 50
nu = 0.5
dx = 1/Nx
dt = nu * dx
ix = np.arange(1, Nx+1)

# 初期条件
x, u = set_initial(Nx, 0)

# プロット
fig, axs = plt.subplots(figsize=(10, 6))
step = 5
for n in range(4):
  plt.plot(x[ix], u[ix], label='t = {:5.3f}'.format(n*step*dt))
  push_ftfs(u, nu, step)

plt.xlim(0.0, 1.0)
plt.xlabel('x')
plt.ylabel('u')
plt.legend(loc='upper left', bbox_to_anchor=(1.0, 1.0))
plt.suptitle('FTFS')

In [ ]:
def push_ftbs(u, nu, step):
  "FTBSスキームによってstep数だけ時間更新する"
  Nx = np.size(u) - 2
  ix = np.arange(1, Nx+1, dtype=np.int32)
  for n in range(step):
    # 更新
    u[ix] = u[ix] - nu*(u[ix] - u[ix-1])
    # 境界条件
    u[   0] = u[Nx]
    u[Nx+1] = u[ 1]

# パラメータ
Nx = 50
nu = 0.5
dx = 1/Nx
dt = nu * dx
ix = np.arange(1, Nx+1)

# 初期条件
x, u = set_initial(Nx, 0)

# プロット
fig, axs = plt.subplots(figsize=(10, 6))
step = 5
for n in range(4):
  plt.plot(x[ix], u[ix], label='t = {:5.3f}'.format(n*step*dt))
  push_ftbs(u, nu, step)

plt.xlim(0.0, 1.0)
plt.xlabel('x')
plt.ylabel('u')
plt.legend(loc='upper left', bbox_to_anchor=(1.0, 1.0))
plt.suptitle('FTBS')

In [ ]:
def push_lf(u, nu, step):
  "Lax-Friedrichsスキームによってstep数だけ時間更新する"
  Nx = np.size(u) - 2
  ix = np.arange(1, Nx+1, dtype=np.int32)
  for n in range(step):
    # 更新
    u[ix] = 0.5*(u[ix+1] + u[ix-1]) - 0.5*nu*(u[ix+1] - u[ix-1])
    # 境界条件
    u[   0] = u[Nx]
    u[Nx+1] = u[ 1]

# パラメータ
Nx = 50
nu = 0.5
dx = 1/Nx
dt = nu * dx
ix = np.arange(1, Nx+1)

# 初期条件
x, u = set_initial(Nx, 0)

# プロット
fig, axs = plt.subplots(figsize=(10, 6))
step = 5
for n in range(4):
  plt.plot(x[ix], u[ix], label='t = {:5.3f}'.format(n*step*dt))
  push_lf(u, nu, step)

plt.xlim(0.0, 1.0)
plt.xlabel('x')
plt.ylabel('u')
plt.legend(loc='upper left', bbox_to_anchor=(1.0, 1.0))
plt.suptitle('Lax-Friedrichs')

In [ ]:
def push_lw(u, nu, step):
  "Lax-Wendroffスキームによってstep数だけ時間更新する"
  Nx = np.size(u) - 2
  ix = np.arange(1, Nx+1, dtype=np.int32)
  for n in range(step):
    # 更新
    u[ix] = u[ix] - \
      0.5*nu*(u[ix+1] - u[ix-1]) + \
      0.5*nu**2*(u[ix+1] - 2*u[ix] + u[ix-1])
    # 境界条件
    u[   0] = u[Nx]
    u[Nx+1] = u[ 1]

# パラメータ
Nx = 50
nu = 0.5
dx = 1/Nx
dt = nu * dx
ix = np.arange(1, Nx+1)

# 初期条件
x, u = set_initial(Nx, 0)

# プロット
fig, axs = plt.subplots(figsize=(10, 6))
step = 5
for n in range(4):
  plt.plot(x[ix], u[ix], label='t = {:5.3f}'.format(n*step*dt))
  push_lw(u, nu, step)

plt.xlim(0.0, 1.0)
plt.xlabel('x')
plt.ylabel('u')
plt.legend(loc='upper left', bbox_to_anchor=(1.0, 1.0))
plt.suptitle('Lax-Wendroff')

In [ ]:
def push_lw_av(u, nu, step, epsilon):
  "Lax-Wendroffスキーム（人工粘性あり）によってstep数だけ時間更新する"
  Nx = np.size(u) - 2
  ix = np.arange(1, Nx+1, dtype=np.int32)
  for n in range(step):
    # 人工粘性の係数を決定
    kappa = epsilon*np.abs(u[ix+1] - 2*u[ix] + u[ix+1])
    # 更新
    u[ix] = u[ix] - \
      0.5*nu*(u[ix+1] - u[ix-1]) + \
      (0.5*nu**2 + kappa)*(u[ix+1] - 2*u[ix] + u[ix-1])
    # 境界条件
    u[   0] = u[Nx]
    u[Nx+1] = u[ 1]

# パラメータ
Nx = 50
nu = 0.5
dx = 1/Nx
dt = nu * dx
ix = np.arange(1, Nx+1)
epsilon = 0.2

# 初期条件
x, u = set_initial(Nx, 0)

# プロット
fig, axs = plt.subplots(figsize=(10, 6))
step = 5
for n in range(4):
  plt.plot(x[ix], u[ix], label='t = {:5.3f}'.format(n*step*dt))
  push_lw_av(u, nu, step, epsilon)

plt.xlim(0.0, 1.0)
plt.xlabel('x')
plt.ylabel('u')
plt.legend(loc='upper left', bbox_to_anchor=(1.0, 1.0))
plt.suptitle('Lax-Wendroff with Artificial Viscosity')